<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colab-env --quiet

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive


In [2]:
#!pip install huggingface_hub --quiet
from huggingface_hub import HfApi

api = HfApi()
api.get_token_permission(token=access_token_write)
#api.set_access_token(access_token)


# frankmorales2020/Mistral-7B-text-to-sql Good

repo_id = "my-awesome-model-poc"
#api.create_repo(repo_id=repo_id, private=False)

#api.delete_repo(repo_id=repo_id)

#api.upload_folder(
#    folder_path="./model",
#    repo_id=repo_id,
#    repo_type="model",
#)


In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [ ]:
#frankmorales2020/Mistral-7B-text-to-sql
# Chose the base model you want
#model_name = "frankmorales2020/Mistral-7B-text-to-sql" # 01/03/2023
model_name = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2" # 04/03/2024
# set device
device = 'cuda'
#v Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

In [ ]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

# Load dataset from the hub
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(12500))

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"][345]["messages"])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

In [ ]:
from datasets import load_dataset
from random import randint

# Load our test dataset and Tokenizer again
#tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql") # 01/03/2024
tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2") # 04/03/2024
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

In [8]:
# generate the same prompt as for the first local test
prompt = tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
request= {"inputs":prompt,"parameters":{"temperature":0.2, "top_p": 0.95, "max_new_tokens": 256}}


In [9]:
! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

Parameter-Efficient Fine-Tuning (PEFT)

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql" # 01/03/2024
peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2" # 04/03/2024 and 10/03/2024
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [11]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Query:
What is the average grid for competitors who had more than 22 laps and time/retired of +17.276?
Original Answer:
SELECT AVG(grid) FROM table_name_79 WHERE time_retired = "+17.276" AND laps > 22
Generated Answer:
SELECT AVG(grid) FROM table_name_79 WHERE time_retired = "+17.276" AND laps > 22


https://huggingface.co/frankmorales2020

In [12]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    print()
    print()
    print('Question: %s\n'%sample["messages"][1]["content"])
    #print(sample["messages"][2]["content"])
    #print('Predicted Answer: %s'%sample["messages"][2]["content"])
    print()
    if predicted_answer == sample["messages"][2]["content"]:
        #print('Success!')
        print('Success Answer: %s'%sample["messages"][2]["content"])
        return 1
    else:
        print('Real Answer: %s'%predicted_answer)
        print('Failed Answer: %s'%sample["messages"][2]["content"])
        return 0

success_rate = []

number_of_eval_samples = 10
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

 10%|█         | 1/10 [00:01<00:14,  1.67s/it]



Question: Where did arturo segovia compete?


Success Answer: SELECT location FROM table_name_24 WHERE opponent = "arturo segovia"


 20%|██        | 2/10 [00:04<00:19,  2.45s/it]



Question: What is the status of the community that has an area larger than 5.2 sq km, and a Census Ranking of 2,531 of 5,008?


Success Answer: SELECT status FROM table_name_60 WHERE area_km_2 > 5.2 AND census_ranking = "2,531 of 5,008"


 30%|███       | 3/10 [00:06<00:15,  2.17s/it]

...



Question: What was the drawn amount for teams with points of 12 and played smaller than 10?


Real Answer: SELECT SUM(drawn) FROM table_name_13 WHERE points = 12 AND played < 10
Failed Answer: SELECT AVG(drawn) FROM table_name_13 WHERE points = 12 AND played < 10


 90%|█████████ | 9/10 [00:17<00:01,  1.92s/it]



Question: How many drafts featured fred hoaglin?


Success Answer: SELECT COUNT(overall) FROM table_name_42 WHERE player = "fred hoaglin"


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 10/10 [00:20<00:00,  2.01s/it]



Question: Name the ticket/office for orville preston


Success Answer: SELECT ticket___office FROM table_22607062_1 WHERE secretary_of_state = "Orville Preston"
Accuracy: 80.00%


When evaluated on 10 samples from the evaluation dataset, our model achieved an impressive accuracy of 80.00%.


In [13]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    if predicted_answer == sample["messages"][2]["content"]:
        return 1
    else:
        return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

100%|██████████| 1000/1000 [33:46<00:00,  2.03s/it]

Accuracy: 80.90%



When evaluated on 1000 samples from the evaluation dataset, our model achieved an impressive accuracy of 80.90%. However, there's room for improvement. We could enhance the model's performance by exploring techniques like few-shot learning, RAG, and Self-healing to generate the SQL query.